In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [ ]:
# Think to try to improve: 
# 1. What happens if you use larger network
# 2. Better normalization with BatchNorm
# 3. Different learning rate (is there a better one)?
# 4. Change architecture to a CNN

In [2]:
class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128, 1), # 1 or 0 (Discriminator)
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.disc(x)

In [3]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim), # 28x28x1 -> 784
            nn.Tanh(), # between -1 and 1
        )

    def forward(self, x):
        return self.gen(x)

In [4]:
# Hyperparameters etc.
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64 # 128, 256
image_dim = 28 * 28 * 1 # 784
batch_size = 32
num_epochs = 50

In [5]:
disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)

In [6]:
transforms = transforms.Compose(
    # [transforms.ToTensor(), transforms.Normalize((0.1307, ), (0.3081, ))]
    [transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5, ))]
)

dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step = 0

In [8]:
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(tqdm(loader)):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]

        ### Train Discriminator: max log(D(real)) + log(1 - D(G(z))) # z is random noise
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1) # -1 means to flatten
        # disc_fake = disc(fake.detach()).view(-1) # .detach() -> when we run backward pass, we don't clear those intermediate computation
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward(retain_graph=True) # retain_graph=True -> alternative to fake.detach()
        opt_disc.step()

        ### Train Generator min log(1 - D(G(z))) [slower training] <-> max log(D(G(z))) [better]
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] "
                f"Loss D: {lossD:.4f}, Loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )

                writer_fake.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )

                step += 1

  0%|                                                                                 | 1/1875 [00:00<17:38,  1.77it/s]

Epoch [0/50] Loss D: 0.6747, Loss G: 0.6898


  0%|▍                                                                                | 9/1875 [00:00<00:40, 46.18it/s]

Epoch [1/50] Loss D: 0.7606, Loss G: 0.6442


  1%|▌                                                                               | 13/1875 [00:00<00:28, 65.07it/s]

...

  1%|▌                                                                               | 12/1875 [00:00<00:32, 56.95it/s]

Epoch [48/50] Loss D: 0.7151, Loss G: 0.8898


  0%|▎                                                                                | 8/1875 [00:00<00:46, 40.34it/s]

Epoch [49/50] Loss D: 0.6640, Loss G: 1.0886


100%|██████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:31<00:00, 60.07it/s]
